In [1]:
%load_ext autoreload
%autoreload 2

In [36]:
import numpy as np
from ML.credit_risk.pipeline import data_train_pipeline
from ML.credit_risk.pipeline.data_operations import split_train_data, f1_score
from ML.credit_risk.pipeline.read_data import read_data_train

from xgboost import XGBClassifier

In [15]:
data_original = read_data_train()

In [16]:
data = data_train_pipeline.fit_transform(data_original.copy(deep=True))

In [20]:
data["TARGET"].value_counts()

0    140910
1     10540
Name: TARGET, dtype: int64

In [21]:
140910/10540

13.369070208728653

In [29]:
x_train, x_test, y_train, y_test = split_train_data(data, ["TARGET"], 42)

In [28]:
booster = XGBClassifier(
    max_depth = 5, 
    learning_rate = 1, 
    scale_pos_weight = 140910/10540, 
    objective = 'binary:logistic'
)

In [30]:
booster.fit(x_train, y_train)

D:\coding\Jupyter_notebooks\envML\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
D:\coding\Jupyter_notebooks\envML\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
[22:27:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=2, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=13.369070208728653,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [31]:
f1_score(y_test, booster.predict(x_test))

true positive: 1061
true negative: 21199
false positive: 6983
false negative: 1047
D:\coding\Jupyter_notebooks\envML\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


0.20902285263987389

In [32]:
f1_score(y_train, booster.predict(x_train))

D:\coding\Jupyter_notebooks\envML\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
true positive: 7141
true negative: 87660
false positive: 25068
false negative: 1291


0.35141851824512194

In [34]:
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(booster, x_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)

NameError: name 'mean' is not defined

In [37]:
np.mean(scores)

0.6754671077171199

In [38]:
scores

array([0.67875504, 0.67519101, 0.67829535, 0.65236697, 0.68825427,
       0.69499667, 0.66986582, 0.67647369, 0.68779947, 0.67408327,
       0.6890086 , 0.67935553, 0.66620908, 0.67907304, 0.6778626 ,
       0.67094367, 0.6597599 , 0.6756377 , 0.66350248, 0.68901363,
       0.66354028, 0.68198924, 0.67809884, 0.68340272, 0.67969131,
       0.67083787, 0.65324773, 0.67048193, 0.68574787, 0.67052766])